In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import tensorflow as tf

from tensorflow.examples.tutorials.mnist import input_data

In [2]:
flags = tf.app.flags
FLAGS = flags.FLAGS
flags.DEFINE_boolean('fake_data', False, 'If true, uses fake data '
                     'for unit testing.')
flags.DEFINE_integer('max_steps', 1000, 'Number of steps to run trainer.')
flags.DEFINE_float('learning_rate', 0.01, 'Initial learning rate.')

In [3]:
# Import data
mnist = input_data.read_data_sets('data/', one_hot=True,
                                fake_data=FLAGS.fake_data)

sess = tf.InteractiveSession()

# Create the model
x = tf.placeholder(tf.float32, [None, 784], name='x-input')
W = tf.Variable(tf.zeros([784, 10]), name='weights')
b = tf.Variable(tf.zeros([10], name='bias'))

# Use a name scope to organize nodes in the graph visualizer
with tf.name_scope('Wx_b'):
    y = tf.nn.softmax(tf.matmul(x, W) + b)

# Add summary ops to collect data
_ = tf.histogram_summary('weights', W)
_ = tf.histogram_summary('biases', b)
_ = tf.histogram_summary('y', y)

# Define loss and optimizer
y_ = tf.placeholder(tf.float32, [None, 10], name='y-input')
# More name scopes will clean up the graph representation
with tf.name_scope('xent'):
    cross_entropy = -tf.reduce_sum(y_ * tf.log(y))
    _ = tf.scalar_summary('cross entropy', cross_entropy)
with tf.name_scope('train'):
    train_step = tf.train.GradientDescentOptimizer(
        FLAGS.learning_rate).minimize(cross_entropy)

with tf.name_scope('test'):
    correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    _ = tf.scalar_summary('accuracy', accuracy)

# Merge all the summaries and write them out to /tmp/mnist_logs
merged = tf.merge_all_summaries()
writer = tf.train.SummaryWriter('mnist_logs', sess.graph_def)
tf.initialize_all_variables().run()

# Train the model, and feed in test data and record summaries every 10 steps

for i in range(FLAGS.max_steps):
    if i % 10 == 0:  # Record summary data and the accuracy
        if FLAGS.fake_data:
            batch_xs, batch_ys = mnist.train.next_batch(
                100, fake_data=FLAGS.fake_data)
            feed = {x: batch_xs, y_: batch_ys}
        else:
            feed = {x: mnist.test.images, y_: mnist.test.labels}
        result = sess.run([merged, accuracy], feed_dict=feed)
        summary_str = result[0]
        acc = result[1]
        writer.add_summary(summary_str, i)
        print('Accuracy at step %s: %s' % (i, acc))
    else:
        batch_xs, batch_ys = mnist.train.next_batch(
          100, fake_data=FLAGS.fake_data)
        feed = {x: batch_xs, y_: batch_ys}
        sess.run(train_step, feed_dict=feed)
print(accuracy.eval({x: mnist.test.images, y_: mnist.test.labels}))

Extracting data/train-images-idx3-ubyte.gz
Extracting data/train-labels-idx1-ubyte.gz
Extracting data/t10k-images-idx3-ubyte.gz
Extracting data/t10k-labels-idx1-ubyte.gz
Accuracy at step 0: 0.098
Accuracy at step 10: 0.7404
Accuracy at step 20: 0.8041
Accuracy at step 30: 0.814
Accuracy at step 40: 0.8421
Accuracy at step 50: 0.8381
Accuracy at step 60: 0.8518
Accuracy at step 70: 0.8403
Accuracy at step 80: 0.8066
Accuracy at step 90: 0.8624
Accuracy at step 100: 0.871
Accuracy at step 110: 0.8864
Accuracy at step 120: 0.8973
Accuracy at step 130: 0.8957
Accuracy at step 140: 0.8817
Accuracy at step 150: 0.8612
Accuracy at step 160: 0.8835
Accuracy at step 170: 0.8527
Accuracy at step 180: 0.8993
Accuracy at step 190: 0.9012
Accuracy at step 200: 0.8909
Accuracy at step 210: 0.9038
Accuracy at step 220: 0.8846
Accuracy at step 230: 0.8921
Accuracy at step 240: 0.9017
Accuracy at step 250: 0.901
Accuracy at step 260: 0.8994
Accuracy at step 270: 0.9067
Accuracy at step 280: 0.9004
Accu